In [1]:
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Dense
from keras import regularizers

import numpy as np
# fix random seed for reproducibility
np.random.seed(42)

import pandas as pd

Using TensorFlow backend.


In [2]:
#Folder for the dataset
datasetFolder = '/home/carnd/dbpedia2016/inRelFinal_1xall/dataset/'

#Number of files
numberOfFiles = 640

#Test split
testSplit=0.25

In [3]:
def load_data(datasetFolder, datasetXFile, datasetYFile, wrap=False):
    #print('Loading {} & {}'.format(datasetXFile, datasetYFile))
    # load file
    with open(datasetFolder + datasetXFile, "r") as f:
        head = f.readline()
        cols = head.split(',')
        numberOfCols = len(cols)
        #print(numberOfCols)
        numberOfRows=0
        for line in f:
            numberOfRows+=1
        f.close()
    
    #print('{} x {}'.format(numberOfRows,numberOfCols))
    if(wrap==True):
        maxY = 1000
    else:
        maxY = numberOfCols-1
    dataX = np.zeros([numberOfRows,maxY],np.int8)
    with open(datasetFolder + datasetXFile, "r") as f:
        head = f.readline()
        rowCounter=0
        for line in f:
            row=line.split(', ')
            for i in range(1, len(row)):
                if(int(row[i])<=0):
                    continue;
                dataX[rowCounter][(int(row[i])-1)%maxY]=1 + ((int(row[i])-1)//maxY)
                #if((1 + ((int(row[i])-1)//maxY))>1):
                #    print("{} data[{}][{}] = {}".format(int(row[i])-1, rowCounter,(int(row[i])-1)%maxY,1 + ((int(row[i])-1)//maxY)))
            rowCounter+=1
        f.close()
        
    dataY=pd.read_csv(datasetFolder + datasetYFile)

    # delete the id fields
    #del dataX['id']
    del dataY['id']

    return dataX, dataY.as_matrix()

In [4]:
dataX, dataY = load_data(datasetFolder,'datasetX_1.csv', 'datasetY_1.csv',wrap=False)

In [5]:
print(dataX.shape)
print(dataX[0:5])

(1167, 1144)
[[0 0 0 ..., 0 0 0]
 [0 0 1 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [6]:
print(dataY.shape)
print(dataY[0:5])

(1167, 527)
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [7]:
print("Input Features: {}".format(dataX.shape[1]))
print("Output Classes: {}".format(dataY.shape[1]))

Input Features: 1144
Output Classes: 527


In [8]:
# create model
logisticRegression = Sequential(name='Simple Logistic Regression')
logisticRegression.add(Dense(dataY.shape[1], input_dim=dataX.shape[1], activation='sigmoid', init='glorot_uniform'))

# simpleModel = Sequential(name='2 Fully Connected Layers')
# simpleModel.add(Dense(1024, input_dim=dataX.shape[1], activation='relu', init='glorot_uniform'))
# simpleModel.add(Dense(dataY.shape[1], activation='sigmoid', init='glorot_uniform'))

deepModel = Sequential(name='Deep Model (5 Dense Layers)')
deepModel.add(Dense(2048, input_dim=dataX.shape[1], activation='relu', init='glorot_uniform'))
deepModel.add(Dropout(0.2))
deepModel.add(Dense(2048, activation='elu', init='glorot_uniform'))
deepModel.add(Dense(1024, activation='relu', init='glorot_uniform'))
deepModel.add(Dropout(0.2))
deepModel.add(Dense(1024, activation='elu', init='glorot_uniform'))
deepModel.add(Dense(768, activation='relu', init='glorot_uniform'))
deepModel.add(Dropout(0.2))
deepModel.add(Dense(768, activation='elu', init='glorot_uniform'))
deepModel.add(Dense(dataY.shape[1], activation='sigmoid', init='glorot_uniform'))

models = [logisticRegression, deepModel]

In [9]:
# Compile model
import keras.backend as K

def count_predictions(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives, predicted_positives, possible_positives

def f1score(y_true, y_pred):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = 2.0 * precision * recall / (precision+recall+ K.epsilon())
    return f1score

def fBetaScore(y_true, y_pred, beta):
    true_positives, predicted_positives, possible_positives = count_predictions(y_true, y_pred)
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1score = (1+(beta*beta)) * precision * recall / ((beta*beta*precision)+recall+ K.epsilon())
    return f1score

for model in models:
    model.compile(loss='binary_crossentropy', optimizer='adagrad', metrics=[f1score])

In [10]:
def fit_data(model, dataX, dataY):
    # Fit the model
    #model.fit(dataX, dataY, nb_epoch=5, verbose=2, batch_size=256)
    return model.train_on_batch(dataX, dataY)

In [11]:
def countPredictions(y_true, y_pred):
    true_positives = np.sum(np.round(y_pred*y_true))
    predicted_positives = np.sum(np.round(y_pred))
    possible_positives = np.sum(y_true)
    return true_positives, predicted_positives, possible_positives

In [12]:
#Randomize the list of numbers so we can split train and test dataset
listOfFiles=list(range(1,numberOfFiles+1))
import random
random.shuffle(listOfFiles)
splitIndex=int((1-testSplit)*numberOfFiles)

numberOfEons = 25
for eon in range(0, numberOfEons):
    print('{}. Eon {}/{}'.format(eon+1,eon+1, numberOfEons))
    for trainIndex in range(0,splitIndex):
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[trainIndex]), 'datasetY_{}.csv'.format(listOfFiles[trainIndex]))
        for model in models:
            #print('Model = {}'.format(model.name))
            #model.fit(dataX, dataY, nb_epoch=5, verbose=0, batch_size=512)
            #sc=model.test_on_batch(dataX,dataY)
            #loss = sc[0]
            #f1score = sc[1]
            loss, f1score=fit_data(model,dataX, dataY)
            print('Learning for file {} / {} : datasetX/Y_{}\t\tloss={:.4f} f1score={:.4f}'.format(trainIndex+1, splitIndex, listOfFiles[trainIndex], loss, f1score), end='\r')
        
    counts = {} 
    for model in models:
        counts[model.name] = {'true_positives':0, 'predicted_positives':0, 'possible_positives':0}
    
    for testIndex in range(splitIndex, numberOfFiles):
        dataX, dataY = load_data(datasetFolder,'datasetX_{}.csv'.format(listOfFiles[testIndex]), 'datasetY_{}.csv'.format(listOfFiles[testIndex]))
        for model in models:
            predY=model.predict_on_batch(dataX)
            true_positives, predicted_positives, possible_positives = countPredictions(dataY, predY)
            counts[model.name]['true_positives'] += true_positives
            counts[model.name]['predicted_positives'] += predicted_positives
            counts[model.name]['possible_positives'] += possible_positives
            print ('Testing for file {} / {} : datasetX/Y_{} - true +ve:{}  pred +ve:{} possible +ve:{}'.format(testIndex+1, numberOfFiles, listOfFiles[testIndex], true_positives,predicted_positives,possible_positives), end='\r')
    
    for model in models:
        count = counts[model.name]
        precision = (count['true_positives'])/(count['predicted_positives']+0.0001)
        recall = (count['true_positives'])/(count['possible_positives']+0.0001)
        f1score = 2.0 * precision * recall / (precision+recall+0.0001)
        print(' - Model = {} \t f1-score = {:.4f}\t precision = {:.4f} \t recall = {:.4f}'.format(model.name, f1score, precision, recall))

1. Eon 1/25
 - Model = Simple Logistic Regression 	 f1-score = 0.2069	 precision = 0.8599 	 recall = 0.11768
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.3471	 precision = 0.7889 	 recall = 0.2225
2. Eon 2/25
 - Model = Simple Logistic Regression 	 f1-score = 0.2263	 precision = 0.8510 	 recall = 0.13054
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.3878	 precision = 0.7913 	 recall = 0.2568
3. Eon 3/25
 - Model = Simple Logistic Regression 	 f1-score = 0.2469	 precision = 0.8481 	 recall = 0.14454
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.3968	 precision = 0.7884 	 recall = 0.2651
4. Eon 4/25
 - Model = Simple Logistic Regression 	 f1-score = 0.2773	 precision = 0.8391 	 recall = 0.16614
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.4076	 precision = 0.7858 	 recall = 0.2752
5. Eon 5/25
 - Model = Simple Logistic Regression 	 f1-score = 0.2848	 precision = 0.8392 	 recall = 0.17154
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.4203	 precisi

==================================================
# ainRelFinal_1xall
==================================================
1. Eon 1/25
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.4251	 precision = 0.7243 	 recall = 0.3009
 - Model = Simple Logistic Regression 	 f1-score = 0.3513	 precision = 0.8005 	 recall = 0.2251
2. Eon 2/25
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.4120	 precision = 0.7618 	 recall = 0.2824
 - Model = Simple Logistic Regression 	 f1-score = 0.3787	 precision = 0.7964 	 recall = 0.2485
3. Eon 3/25
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.4076	 precision = 0.7667 	 recall = 0.2776
 - Model = Simple Logistic Regression 	 f1-score = 0.3857	 precision = 0.7945 	 recall = 0.2547
4. Eon 4/25
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.4153	 precision = 0.7620 	 recall = 0.2855
 - Model = Simple Logistic Regression 	 f1-score = 0.3895	 precision = 0.7941 	 recall = 0.2580
5. Eon 5/25
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.4304	 precision = 0.7418 	 recall = 0.3032
 - Model = Simple Logistic Regression 	 f1-score = 0.3927	 precision = 0.7944 	 recall = 0.2608
6. Eon 6/25
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.4206	 precision = 0.7757 	 recall = 0.2885
 - Model = Simple Logistic Regression 	 f1-score = 0.3939	 precision = 0.7944 	 recall = 0.2619
7. Eon 7/25
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.4234	 precision = 0.7495 	 recall = 0.2951
 - Model = Simple Logistic Regression 	 f1-score = 0.3956	 precision = 0.7930 	 recall = 0.2636
8. Eon 8/25
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.4046	 precision = 0.7662 	 recall = 0.2749
 - Model = Simple Logistic Regression 	 f1-score = 0.3965	 precision = 0.7932 	 recall = 0.2644
9. Eon 9/25
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.4103	 precision = 0.7660 	 recall = 0.2802
 - Model = Simple Logistic Regression 	 f1-score = 0.3963	 precision = 0.7940 	 recall = 0.2641
10. Eon 10/25
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.4112	 precision = 0.7831 	 recall = 0.2789
 - Model = Simple Logistic Regression 	 f1-score = 0.3965	 precision = 0.7941 	 recall = 0.2643
11. Eon 11/25
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.4118	 precision = 0.7681 	 recall = 0.2813
 - Model = Simple Logistic Regression 	 f1-score = 0.3970	 precision = 0.7942 	 recall = 0.2647
12. Eon 12/25
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.3999	 precision = 0.7808 	 recall = 0.2688
 - Model = Simple Logistic Regression 	 f1-score = 0.3977	 precision = 0.7939 	 recall = 0.2653
13. Eon 13/25
 - Model = Deep Model (5 Dense Layers) 	 f1-score = 0.4087	 precision = 0.7533 	 recall = 0.2804
 - Model = Simple Logistic Regression 	 f1-score = 0.3989	 precision = 0.7920 	 recall = 0.2666